# 気象データを用いた熱中症患者数の予測

---

## 1. はじめに

本ノートブックでは、気象データから東京の熱中症患者数を予測する機械学習モデルの構築を行う。

機械学習のアルゴリズムには、重回帰分析を利用する。


## 2. 本ノートブックの目的

本ノートブックの目的は、以下の３つの処理の実行と検証

- モデルの学習
- 学習済みモデルの検証
- 学習済みモデルの保存

---

## 3. 処理フロー

1. データセットのロード
2. データセットの確認
3. 重回帰分析の学習
4. 学習済みモデルの検証
5. 学習済みモデルの保存

<br>

---

## 4. 処理詳細

### 4.1. 必要なライブラリのインポート

本分析の学習に必要なライブラリをまとめて、インポートする

利用するライブラリとその用途を下記にまとめる


|パッケージ名|用途|
|---|:--|
|pandas|データフレームの使用|
|train_test_split|訓練データとテストデータの分割|
|LinearRegression|重回帰分析の実装|
|cross_val_score|K-分割交差検証|
|numpy|VIFの計算|
|variance_inflation_factor|VIFの計算|
|pickle|モデルの保存|

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import cross_val_score

import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

import pickle

### 4.2. データフレームの定義

本分析で用いる、データフレームを定義する

|データ名|用途|
|:-:|:--|
|data_merge|熱中症情報の結合|
|data_weather|熱中症情報と気象データの結合|

In [2]:
data_merge = pd.DataFrame()
data_weather = pd.DataFrame()

### 4.3. データセットのダウンロードと整形

以下、本分析で使用するデータ

|データ名|形式|内容|URL|
|:-:|:--|:--|:--|
|weather_tokyo.csv|Microsoft Excel CSV ファイル (.csv)|気象庁 過去の気象データ|https://www.data.jma.go.jp/risk/obsdl/index.php|
|2010～2022.xlsx|Microsoft Excel ワークシート (.xlsx)|総務省消防庁　熱中症情報|https://www.fdma.go.jp/disaster/heatstroke/post4.html|
|final_wbgt_44132.csv|Microsoft Excel CSV ファイル (.csv)|熱中症予防情報サイト|https://www.wbgt.env.go.jp/wbgt_data.php|

In [3]:
weather = pd.read_csv('weather_tokyo.csv', encoding='shift_jis')

In [4]:
weather.head() 

,日付,最高気温(℃),日照時間(時間),平均風速(m/s),平均湿度(％)
0,2010-05-01,20.1,11.7,2.7,45
1,2010-05-02,22.1,11.4,2.8,41
2,2010-05-03,24.0,8.9,4.0,51
3,2010-05-04,24.7,7.2,2.9,61
4,2010-05-05,27.6,10.5,2.5,64


#### 熱中症情報について整形を行う

①各シートにアクセス

②都道府県コードが13（東京）であるものをdataに格納

In [5]:
year = ['2010', '2011','2012','2013', '2014', '2015', '2016', '2017','2018', '2019', '2020', '2021','2022']

for y in year:
    df = pd.read_excel(y+'.xlsx', sheet_name=None)
    
    for i in range(6,10):
        data = df[y+'_0'+str(i)] #①
        data = data[data['都道府県コード'] == 13] #②
        data_merge = pd.concat([data_merge, data])
        
data_merge = data_merge[['日付', '搬送人員（計）']]

In [6]:
data_merge.head()

,日付,搬送人員（計）
12,2010-06-01,0.0
56,2010-06-02,1.0
100,2010-06-03,1.0
145,2010-06-04,3.0
191,2010-06-05,2.0


### 4.4. データの選定

|データ名|カラム名|用途|
|:--|:-:|:-:|
|2010～2022.xlsx|日付|外部キー|
|2010～2022.xlsx|搬送人員（計）|目的関数|

本分析では最高気温が28を超えるものだけで分析する

 - 参考文献　https://www.wbgt.env.go.jp/wbgt.php

In [7]:
weather = weather[weather['最高気温(℃)'] > 28]

data_merge['日付'] =  data_merge['日付'].astype(str)

data_merge = data_merge.dropna()

### 4.5. 説明変数の追加

搬送人員との相関係数が高いため体感温度を説明変数として追加。


ミスナールの計算式(改良版)
<br>

$$ 37 - \frac{37 -t}{0.68 - 0.0014 × h + \frac{1}{a}} - 0.29t × (1 - \frac{h}{100}) $$

$$ t：気温（℃）、h：湿度（％）、v：風速（m/s）、a： = 1.76 + 1.4v^{0.75} $$

 - 参考文献　https://keisan.casio.jp/exec/system/1257417058

In [8]:
def taikan(t, h, v):
    a = 1.76 + 1.4*(v**0.75)
    tn = 37 - (37 - t) / (0.68 - 0.0014 * h + 1/a) - 0.29 * t * (1 - h/100)
    return tn

In [9]:
weather['体感温度'] = taikan(weather['最高気温(℃)'], weather['平均湿度(％)'], weather['平均風速(m/s)'])

### 4.6. データの結合と分割

日付を外部キーとして結合

In [10]:
data_weather = pd.merge(data_merge, weather, on='日付')

data_weather = data_weather.drop(['日付','平均風速(m/s)','最高気温(℃)','平均湿度(％)'],axis=1)

①目的変数y、説明変数xに分割

|変数名|使用データ|
|:-:|:-:|
|y|搬送人員|
|x|体感温度　日照時間|

In [11]:
y = data_weather['搬送人員（計）'].values
x = data_weather.drop(labels=['搬送人員（計）'], axis=1).values

②訓練データ、テストデータをそれぞれ、7:3に分割する

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

### 4.6. 重回帰分析の実装


In [13]:
model = LinearRegression()

model.fit(x_train, y_train)

LinearRegression()

## 5. 分析結果と検証

### 5.1. 重回帰分析の精度


In [14]:
print('重回帰分析')
print('train score : ', model.score(x_train, y_train))
print('test score : ', model.score(x_test, y_test))

重回帰分析
train score :  0.5690448385430392
test score :  0.5538601980987812


### 5.2. K-分割交差検証

K-分割交差検証　データをK個に分割してそのうち1つをテストデータに残りのK-1個を学習データとして正解率の評価を行う手法

本分析では6個に分割して評価を行う

In [15]:
scores = cross_val_score(model, x, y, cv=6, scoring='r2')

print("各分割のスコア:", scores)
print("平均スコア:", np.mean(scores))

各分割のスコア: [0.59456462 0.39327297 0.71672012 0.51859043 0.56645033 0.4868164 ]
平均スコア: 0.5460691445313347


### 5.3. 多重共線性

多重共線性　説明変数間の相関が高い時又は，ある説明変数が他の説明変数によって説明されてしまう時などに起こる

本分析では分散拡大要因(Variance Inflation Factor: VIF)という指標を用いて，VIFが10以上になっていないか確認する

In [16]:
vif = pd.DataFrame()
vif["Variable"] = data_weather.columns
vif["VIF"] = [variance_inflation_factor(data_weather.values, i) for i in range(data_weather.shape[1])]

vif.sort_values('VIF', ascending=False)

,Variable,VIF
1,日照時間(時間),5.038498
2,体感温度,4.641181
0,搬送人員（計）,2.232179


### 5.4. 学習済みモデルの保存

In [17]:
with open('model.pickle', mode='wb') as f:
    pickle.dump(model,f,protocol=2)

## 6. 改善点

1. 天候以外のデータの利用

熱中症の影響は気象データだけでなく、地形、人口、都市環境などの多くの要素が影響すると考えられる。これらの要素を組み合わせた総合的な分析を行い、熱中症のリスク要因や影響を包括的に評価し、効果的な予防策や対策を検討することが可能。

2. 詳細な搬送人員のデータ

本分析では都道府県単位の搬送人員データを使用しましたが、さらに詳細な地域別や時間帯別の搬送人員データを取得・活用することで、熱中症の発生パターンや影響の時間的変化を捉えることが可能である。これにより、より精緻な分析と熱中症のリスク予測が行える。